In [1]:
import sys
from pathlib import Path
import pandas as pd

In [2]:
# c4を混ぜる割合に関わらず、framenetの分割には違いがないことを確認

pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 1000)

for add_method in ["sequential", "ratio"]:
    print(add_method)
    for part in ["all_3_0", "all_3_1", "all_3_2"]:
        print(part)
        for split in ["dev", "test", "train"]:
            print(split)
            input_file0 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/0/{part}/exemplars_{split}.jsonl")
            input_file1 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/1/{part}/exemplars_{split}.jsonl")
            input_file2 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/2/{part}/exemplars_{split}.jsonl")

            df0 = pd.read_json(input_file0, lines=True)
            df1 = pd.read_json(input_file1, lines=True)
            df2 = pd.read_json(input_file2, lines=True)
            print(len(df0), len(df1), len(df2))

            df0 = df0[df0["source"] == "framenet"]
            df1 = df1[df1["source"] == "framenet"]
            df2 = df2[df2["source"] == "framenet"]
            df0 = df0[["ex_idx","verb"]]
            df1 = df1[["ex_idx","verb"]]
            df2 = df2[["ex_idx","verb"]]

            print(len(df0), len(df1), len(df2))

            print(pd.merge(df0, df1, on="ex_idx", how="outer", indicator=True).query('_merge != "both"')["verb_x"].unique())
            print(pd.merge(df1, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"')["verb_x"].unique())
            print(pd.merge(df0, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"')["verb_x"].unique())
            print()
        print()
    print()



sequential
all_3_0
dev


/tmp/ipykernel_1686002/2613218836.py:16: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df0 = pd.read_json(input_file0, lines=True)


ValueError: Expected object or value

In [3]:

# c4の用例について、混ぜる割合が大きいものは混ぜる割合が小さいものを包含していることを確認

pd.set_option('display.max_rows', 10)

for add_method in ["sequential", "ratio"]:
    print(add_method)
    for part in ["all_3_0", "all_3_1", "all_3_2"]:
        print(part)
        for split in ["dev", "test", "train"]:
            print(split)
            input_file0 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/0/{part}/exemplars_{split}.jsonl")
            input_file1 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/1/{part}/exemplars_{split}.jsonl")
            input_file2 = Path(f"../data/verb_clustering_c4/dataset/{add_method}/2/{part}/exemplars_{split}.jsonl")

            df0 = pd.read_json(input_file0, lines=True)
            df1 = pd.read_json(input_file1, lines=True)
            df2 = pd.read_json(input_file2, lines=True)
            print(len(df0), len(df1), len(df2))

            df0 = df0[df0["source"] == "c4"]
            df1 = df1[df1["source"] == "c4"]
            df2 = df2[df2["source"] == "c4"]
            df0 = df0[["ex_idx","verb"]]
            df1 = df1[["ex_idx","verb"]]
            df2 = df2[["ex_idx","verb"]]

            print(len(df0), len(df1), len(df2))

            print(pd.merge(df0, df1, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            print(pd.merge(df1, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            print(pd.merge(df0, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            print()
        print()
    print()

sequential
all_3_0
dev
27644 27644 27644
0 0 0
Empty DataFrame
Columns: [ex_idx, verb_x, verb_y, _merge]
Index: []
Empty DataFrame
Columns: [ex_idx, verb_x, verb_y, _merge]
Index: []
Empty DataFrame
Columns: [ex_idx, verb_x, verb_y, _merge]
Index: []

test
27147 54294 81441
0 27147 54294
                                     ex_idx verb_x   verb_y      _merge
0      000107be-3c57-42bb-932c-e519db2cbcfb    NaN     need  right_only
1      0001cd43-7fe1-449a-b3e1-c4d3477dd0a0    NaN     want  right_only
2      00037f07-26a2-4df3-b097-54a96e231f37    NaN   attend  right_only
3      000668c7-3806-434f-9092-42daddbe4639    NaN     know  right_only
4      00067a45-8220-4105-93f1-93c0bbd88495    NaN  address  right_only
...                                     ...    ...      ...         ...
27142  fff143e1-6841-4d63-a00a-55c8baf4d924    NaN     look  right_only
27143  fff3368d-d4df-447f-8579-80478a1a176a    NaN     take  right_only
27144  fff47346-7707-4131-b3b8-7b3a25fb6682    NaN      say  ri

In [4]:
# 分割したデータセットの内訳を調べる

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

parts = ["all_3_0", "all_3_1", "all_3_2"]
c4_rates = [1, 2]
for part in parts:
    print(f"part: {part}")
    for c4_rate in c4_rates:
        print(f"c4_rate: {c4_rate}")
        input_file = Path(f"../data/verb_clustering_c4/dataset/ratio/{c4_rate}/{part}/exemplars_test.jsonl")

        df = pd.read_json(input_file, lines=True)

        total_count = len(df)
        framenet_count = len(df[df['source'] == 'framenet'])
        c4_count = len(df[df['source'] == 'c4'])
        print(f"1 - (total / (framenet * (1 + c4_rate))):{1 - (total_count / (framenet_count * (1 + c4_rate)))}")
        print(f"total: {total_count} c4: {c4_count} framenet: {framenet_count} * {c4_rate} = {framenet_count * c4_rate}")

        count_df = df.groupby(['lu_name', 'source']).size().unstack(fill_value=0)
        # framenetよりc4が少ないlu_nameをフィルタリング
        less_c4 = count_df[count_df['c4'] < count_df['framenet'] * c4_rate].copy()

        # 差を計算
        less_c4.loc[:,'difference'] = less_c4['framenet'] * c4_rate - less_c4['c4']

        # 結果を表示
        # print(less_c4[['c4', 'framenet', 'difference']])
        print()


part: all_3_0
c4_rate: 1
1 - (total / (framenet * (1 + c4_rate))):0.0013629498655468586
total: 54220 c4: 27073 framenet: 27147 * 1 = 27147

c4_rate: 2
1 - (total / (framenet * (1 + c4_rate))):0.0031679375253250708
total: 81183 c4: 54036 framenet: 27147 * 2 = 54294

part: all_3_1
c4_rate: 1
1 - (total / (framenet * (1 + c4_rate))):0.008356243669512375
total: 54826 c4: 27182 framenet: 27644 * 1 = 27644

c4_rate: 2
1 - (total / (framenet * (1 + c4_rate))):0.01349298220228623
total: 81813 c4: 54169 framenet: 27644 * 2 = 55288

part: all_3_2
c4_rate: 1
1 - (total / (framenet * (1 + c4_rate))):0.0011154806019814112
total: 52833 c4: 26387 framenet: 26446 * 1 = 26446

c4_rate: 2
1 - (total / (framenet * (1 + c4_rate))):0.0030376364415538593
total: 79097 c4: 52651 framenet: 26446 * 2 = 52892



In [5]:
# 既存研究のデータセットとの比較

pd.set_option('display.max_rows', 10)


for add_method in ["sequential", "ratio"]:
    print(add_method)
    for part in ["all_3_0", "all_3_1", "all_3_2"]:
        print(part)
        for split in ["dev", "test", "train"]:
            print(split)
            input_file_yamada_raw = Path(f"../data/preprocessing_framenet/stanza/exemplars.jsonl")
            input_file_yamada = Path(f"../data/verb_clustering/dataset/{part}/exemplars_{split}.jsonl")
            input_file_tsujimoto_raw = Path(f"../data/preprocessing/framenet/preprocess/exemplars.jsonl")
            input_file_tsujimoto = Path(f"../data/verb_clustering_c4/dataset/{add_method}/0/{part}/exemplars_{split}.jsonl")

            df_yamada_raw = pd.read_json(input_file_yamada_raw, lines=True)
            df_yamada = pd.read_json(input_file_yamada, lines=True)

            df_tsujimoto_raw = pd.read_json(input_file_tsujimoto_raw, lines=True)
            df_tsujimoto_raw = df_tsujimoto_raw[["ex_idx", "id_data","frame_id","lu_id"]]
            df_tsujimoto_raw["ex_id"] = df_tsujimoto_raw["id_data"].apply(lambda x: x["id"])
            print(df_tsujimoto_raw.groupby(["ex_id", "id_data","frame_id","lu_id"]).size())
            df_tsujimoto = pd.read_json(input_file_tsujimoto, lines=True)
            print(len(df_yamada), len(df_tsujimoto))


            unique_yamada = set(df_yamada["verb"].unique())
            unique_tsujimoto = set(df_tsujimoto["verb"].unique())

            # 比較
            only_in_yamada = unique_yamada - unique_tsujimoto
            only_in_tsujimoto = unique_tsujimoto - unique_yamada
            common_values = unique_yamada & unique_tsujimoto

            print("df_yamadaにのみ存在する値:", only_in_yamada)
            print("df_tsujimotoにのみ存在する値:", only_in_tsujimoto)
            print("両方に存在する値:", common_values)



            # print(pd.merge(df0, df1, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            # print(pd.merge(df1, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            # print(pd.merge(df0, df2, on="ex_idx", how="outer", indicator=True).query('_merge != "both"'))
            print()
        print()
    print()


sequential
all_3_0
dev


TypeError: unhashable type: 'dict'